In [1]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [2]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [22]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [23]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
raw_datasets

DatasetDict({
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [11]:
raw_datasets['train'][1]

{'translation': {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}}

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [13]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
tokenizer("Hello,I am Geerath")

{'input_ids': [12110, 2, 389, 489, 15642, 581, 10947, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer("Hello, this is a sentence!, ""This is another sentence.")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 2, 239, 23, 414, 8800, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "hi"

In [17]:
def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length = max_input_length, truncation = True)
    
    #setup the tokenizer for target
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length = max_target_length, truncation = True)
        
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
preprocess_function(raw_datasets["train"][:2])

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [19]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1660 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [26]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [27]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
epochs = 10

In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model, return_tensors = "tf")

In [29]:
generation_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors = "tf",pad_to_multiple_of=128)

In [30]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size = batch_size,
    shuffle = True,
    collate_fn = data_collator,
)

In [31]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size = batch_size,
    shuffle = False,
    collate_fn = data_collator,
)

In [32]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size = 8,
    shuffle = False,
    collate_fn = generation_data_collator,
)

In [33]:
optimizer = AdamWeightDecay(learning_rate = learning_rate,weight_decay_rate = weight_decay)
model.compile(optimizer = optimizer)

In [34]:
model.save_pretrained("tf_model/")

In [35]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [36]:
input_text = "My name is Geerath, I am a Machine Learning Engineer"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   500   179  2297   428     5     2   104    38  6858  4461 11447
     57   153   254     0]], shape=(1, 16), dtype=int32)


In [37]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens = True))

मेरा नाम गीतर है, मैं एक मशीन सीखने इंजन कर रहा हूँ


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
